In [1]:
import pandas as pd
df=pd.read_csv("azureml://subscriptions/4f93438e-847c-469d-94ae-ef5ee130ccab/resourcegroups/project_team14/workspaces/mlnotebook/datastores/testml/paths/Final_Data/part-00000-tid-1107757838746000038-dd6987f9-5526-4378-b9fd-633c8f0e2565-1388-1-c000.csv")
df.head()

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
0,Papaya,Goa,North Goa,Mapusa,Papaya,Fruits,1.33,2400.0,3600.0,3000.0,2003-04-01
1,Papaya,Jammu and Kashmir,Jammu,Narwal Jammu (F&V),Other,Fruits,12.00,1150.0,1250.0,1200.0,2003-04-01
2,Papaya,NCT of Delhi,Delhi,Shahdara,Other,Fruits,0.10,150.0,300.0,250.0,2003-04-02
3,Papaya,Punjab,Bhatinda,Bathinda,Other,Fruits,4.50,400.0,450.0,425.0,2003-04-01
4,Papaya,Punjab,Ferozpur,Firozepur City,Other,Fruits,10.00,600.0,800.0,700.0,2003-04-02


In [3]:
!pip install tensorflow


In [4]:

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler



# Assuming 'Reported Date' is a datetime column, convert it to datetime
df['Reported Date'] = pd.to_datetime(df['Reported Date'])

# Randomly shuffle the data
data = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into 5 equal samples
data_samples = []
split_size = len(data) // 5
for i in range(5):
    start_index = i * split_size
    end_index = (i + 1) * split_size if i < 4 else None
    data_samples.append(data.iloc[start_index:end_index])

# Define function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Train LSTM model for each sample
models = []
for i, sample in enumerate(data_samples):
    # Preprocessing: normalize the features
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(sample[['Arrivals (Tonnes)', 'Min Price (Rs/Quintal)', 'Max Price (Rs/Quintal)', 'Modal Price (Rs/Quintal)']])
    X = scaled_data[:, :-1]  # Use all features except 'Modal Price' as input
    y = scaled_data[:, -1]   # Use 'Modal Price' as target

    # Reshape input data to be 3D [samples, timesteps, features]
    X = X.reshape(X.shape[0], 1, X.shape[1])

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)

    # Train the model
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    loss = lstm_model.evaluate(X_test, y_test, verbose=0)
    print(f"Sample {i+1} Loss: {loss}")

    # Save the model
    models.append(lstm_model)


ModuleNotFoundError: No module named 'keras'

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming you have loaded your dataset into a DataFrame called 'df'

# Drop any non-numeric columns or columns that can't be used for modeling
# For instance, if 'Reported Date' isn't relevant for prediction, you might drop it
df_numeric = df.drop(['Commodity', 'State Name', 'District Name', 'Market Name', 'Variety', 'Group', 'Reported Date'], axis=1)

# Encode categorical variables
label_encoder = LabelEncoder()
for column in [ 'State Name', 'District Name', 'Market Name', 'Variety', 'Group']:
    df_numeric[column] = label_encoder.fit_transform(df_numeric[column])

# Split the data into features (X) and target variable (y)
X = df_numeric.drop('Modal Price (Rs/Quintal)', axis=1)
y = df_numeric['Modal Price (Rs/Quintal)']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
predictions = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


KeyError: 'State Name'

In [2]:
dict_com=df['Commodity']

In [4]:
df_dict_com=pd.DataFrame(dict_com.unique())

In [5]:
df_dict_com.count()

0    297
dtype: int64

In [ ]:
df_numeric[column] = label_encoder.fit_transform(df_numeric[column])